<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/exploringdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import numpy as np
import pandas as pd
import string
import time
import urllib.request
import zipfile
import torch
import json

import zipfile
import time

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


from sklearn.linear_model import LogisticRegression



In [2]:
### mount google drive

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


In [3]:
# Unzip the data
datapath = '/content/gdrive/MyDrive/AIPI540_RecProject/'

if not os.path.exists('./data'):
    os.makedirs('./data')

zip_ref = zipfile.ZipFile(datapath+'reviews.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [4]:
zip_ref = zipfile.ZipFile(datapath+'podcasts.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [5]:
zip_ref = zipfile.ZipFile(datapath+'categories.json.zip', 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [ ]:
# podcasts_df = pd.read_json('data/podcasts.json', lines=True)
# categories_df = pd.read_json('data/categories.json', lines=True)
# reviews_df = pd.read_json('data/reviews.json', lines=True)

In [8]:
with open('data/categories.json') as f:
    categories_df = pd.DataFrame(json.loads(line) for line in f)
categories_df.head()

,podcast_id,itunes_id,category
0,943d1c4349913bde09ef82b69f83cfdf,1446474395,education
1,f53f27d03cdc5040ab4fd2ea30e3dd4f,840646770,society-culture
2,49fa5f1097ae9046ce1a0c56e31b533c,1541959455,arts-performing-arts
3,dbe2dc5aa097523aa294a3b659293af4,1146609744,tv-film
4,2349ed9d26f16029cfc69902bf319ca5,1475252651,education


In [9]:
with open('data/podcasts.json') as f:
    podcasts_df = pd.DataFrame(json.loads(line) for line in f)
podcasts_df.head()

,podcast_id,itunes_id,slug,itunes_url,title,author,description,average_rating,ratings_count,scraped_at
0,d3997f1089be9e45fda4113be39765cb,1000000618,christianity-questions-and-answers,https://podcasts.apple.com/us/podcast/christia...,None,None,None,NaN,None,None
1,f201172864310534ae6b6c4f8ae9b8cc,1000016089,rahdo-talks-through,https://podcasts.apple.com/us/podcast/rahdo-ta...,Rahdo Talks Through,noreply@blogger.com (Richard Ham),"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,2019-07-08 06:15:45.106345
2,4e5ce6a47e5d491aac3f588cfb3ece73,1000016800,st-neots-evangelical-church-sermons,https://podcasts.apple.com/us/podcast/st-neots...,None,None,None,NaN,None,None
3,6c476d3dd90c3fe5381153354b326952,1000035657,idiotspeakshow,https://podcasts.apple.com/us/podcast/idiotspe...,IdiotSpeakShow,IdiotSpeakShow,Podcast by IdiotSpeakShow,NaN,None,2019-07-08 06:01:23.683147
4,b9e7fdf5cd545fc8097055e2f2f1d760,1000035702,conciertos-en-el-living,https://podcasts.apple.com/us/podcast/conciert...,None,None,None,NaN,None,None


In [10]:
with open('data/reviews.json') as f:
    reviews_df = pd.DataFrame(json.loads(line) for line in f)
reviews_df.head()

,podcast_id,title,content,rating,author_id,created_at
0,52e3d2c4fab4e80a8bb75ad144671d96,Argh,What the heck is this show? And why can’t I de...,1,a23655ce5565f32,2019-05-01 01:24:01+00
1,52e3d2c4fab4e80a8bb75ad144671d96,Miss gross overrated as interviewer.,The Howard stern interview was horrible. I lea...,1,e5d461c5f91c8d2,2019-06-07 14:25:34+00
2,52e3d2c4fab4e80a8bb75ad144671d96,I love Terri and Fresh Air,People get really pissy about this beautiful f...,5,806cc75442c8f3d,2019-05-15 09:42:53+00
3,52e3d2c4fab4e80a8bb75ad144671d96,Fresh Air subscription changes to Code Switch,I’ve been a subscriber to Fresh Air for over a...,4,3e1e7e8417a3e85,2019-05-06 02:39:10+00
4,52e3d2c4fab4e80a8bb75ad144671d96,Climate change,"Hello, my name is Lily Rudman and I am a fifth...",5,7c680144b582c6d,2019-04-18 00:37:51+00


In [12]:
df = podcasts_df.merge(categories_df, on = ['podcast_id', 'itunes_id'])
df

,podcast_id,itunes_id,slug,itunes_url,title,author,description,average_rating,ratings_count,scraped_at,category
0,d3997f1089be9e45fda4113be39765cb,1000000618,christianity-questions-and-answers,https://podcasts.apple.com/us/podcast/christia...,None,None,None,NaN,None,None,religion-spirituality
1,d3997f1089be9e45fda4113be39765cb,1000000618,christianity-questions-and-answers,https://podcasts.apple.com/us/podcast/christia...,None,None,None,NaN,None,None,christianity
2,f201172864310534ae6b6c4f8ae9b8cc,1000016089,rahdo-talks-through,https://podcasts.apple.com/us/podcast/rahdo-ta...,Rahdo Talks Through,noreply@blogger.com (Richard Ham),"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,2019-07-08 06:15:45.106345,leisure-games
3,f201172864310534ae6b6c4f8ae9b8cc,1000016089,rahdo-talks-through,https://podcasts.apple.com/us/podcast/rahdo-ta...,Rahdo Talks Through,noreply@blogger.com (Richard Ham),"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,2019-07-08 06:15:45.106345,leisure
4,4e5ce6a47e5d491aac3f588cfb3ece73,1000016800,st-neots-evangelical-church-sermons,https://podcasts.apple.com/us/podcast/st-neots...,None,None,None,NaN,None,None,religion-spirituality
...,...,...,...,...,...,...,...,...,...,...,...
3589624,c16669b79e093e976b83a8e61270a903,999971688,save-to-the-uttermost,https://podcasts.apple.com/us/podcast/save-to-...,None,None,None,NaN,None,None,christianity
3589625,b863eaa678d3de4f5721a7492fd32b3e,999975096,the-skift-podcast,https://podcasts.apple.com/us/podcast/the-skif...,The Skift Podcast,Skift,"Skift editors sit down with creatives, executi...",4.5,19,2019-07-07 23:20:30.301705,news-business-news
3589626,920de5c8d1bb6b3fea7d86ac39473d76,999987741,history-of-the-batman,https://podcasts.apple.com/us/podcast/history-...,None,None,None,NaN,None,None,arts
3589627,920de5c8d1bb6b3fea7d86ac39473d76,999987741,history-of-the-batman,https://podcasts.apple.com/us/podcast/history-...,None,None,None,NaN,None,None,arts-books


In [55]:
full_df = df.merge(reviews_df, on='podcast_id')
full_df

,podcast_id,itunes_id,slug,itunes_url,title_x,author,description,average_rating,ratings_count,scraped_at,category,title_y,content,rating,author_id,created_at
0,03a17efdb4a8a11d64d72000a53690e1,1006403754,her-money-matters,https://podcasts.apple.com/us/podcast/her-mone...,Her Money Matters,Jen Hemphill,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,2019-07-08 00:35:30.088304,education-how-to,Podcast Guest Booker/ Writer,This is one of my favorite shows. It's clear t...,5,29dd6a70bd8b3bc,2017-02-24 22:26:09+00
1,03a17efdb4a8a11d64d72000a53690e1,1006403754,her-money-matters,https://podcasts.apple.com/us/podcast/her-mone...,Her Money Matters,Jen Hemphill,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,2019-07-08 00:35:30.088304,education-how-to,Learning Spanish love it!,Love that you are incorporating spanish! Aweso...,5,883f6252ca1c4ef,2018-04-08 18:46:40+00
2,03a17efdb4a8a11d64d72000a53690e1,1006403754,her-money-matters,https://podcasts.apple.com/us/podcast/her-mone...,Her Money Matters,Jen Hemphill,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,2019-07-08 00:35:30.088304,education-how-to,Great show!,Jen does such a great job of talking about mon...,5,9fc0eed0609943c,2015-08-11 21:32:12+00
3,03a17efdb4a8a11d64d72000a53690e1,1006403754,her-money-matters,https://podcasts.apple.com/us/podcast/her-mone...,Her Money Matters,Jen Hemphill,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,2019-07-08 00:35:30.088304,education-how-to,The Heart of Money Matters,I look forwarrd to Jen's podcast every week! J...,5,96cdf2308511db6,2016-08-02 15:37:05+00
4,03a17efdb4a8a11d64d72000a53690e1,1006403754,her-money-matters,https://podcasts.apple.com/us/podcast/her-mone...,Her Money Matters,Jen Hemphill,"Jen Hemphill, Money Confidence Coach goes on a...",4.8,95,2019-07-08 00:35:30.088304,education-how-to,"Informative, Entertaining Money Talk",I love this podcast about money from Jen Hemph...,5,a5e9e57f6162fee,2015-10-28 14:24:25+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,1e46aafcd91e6e46b1af9932776a002d,640440434,all-of-the-above-radio,https://podcasts.apple.com/us/podcast/all-of-t...,All of the Above radio,Hank Thorpe,All of the Above radio: saving the world two h...,5.0,8,2019-07-08 21:25:30.62971,society-culture-personal-journals,Great Music and Conversation,This show has a great mix of music with fun to...,5,fb6ca03240db301,2016-11-03 17:17:06+00
766,1e46aafcd91e6e46b1af9932776a002d,640440434,all-of-the-above-radio,https://podcasts.apple.com/us/podcast/all-of-t...,All of the Above radio,Hank Thorpe,All of the Above radio: saving the world two h...,5.0,8,2019-07-08 21:25:30.62971,society-culture-personal-journals,Full escapism is the truth!,This show is great to take a break from the ca...,5,e3713d1d11c289a,2018-08-19 19:55:01+00
767,1e46aafcd91e6e46b1af9932776a002d,640440434,all-of-the-above-radio,https://podcasts.apple.com/us/podcast/all-of-t...,All of the Above radio,Hank Thorpe,All of the Above radio: saving the world two h...,5.0,8,2019-07-08 21:25:30.62971,society-culture-personal-journals,Hank’s the Man,I’ve known Hank for close to 20 years. He’s an...,5,cb14669ad97547c,2015-07-14 18:08:30+00
768,1e46aafcd91e6e46b1af9932776a002d,640440434,all-of-the-above-radio,https://podcasts.apple.com/us/podcast/all-of-t...,All of the Above radio,Hank Thorpe,All of the Above radio: saving the world two h...,5.0,8,2019-07-08 21:25:30.62971,society-culture-personal-journals,Hank is my dude,Great show. Can't be mad at anything while thi...,5,e1e474835a0be76,2018-01-27 16:11:26+00


In [15]:
df.ratings_count.value_counts()

1       33493
2       19335
3       13199
4        9761
5        7931
        ...  
56K         2
6.2K        1
31K         1
39K         1
19K         1
Name: ratings_count, Length: 191, dtype: int64

In [24]:
df.columns

Index(['podcast_id', 'itunes_id', 'slug', 'itunes_url', 'title', 'author',
       'description', 'average_rating', 'ratings_count', 'scraped_at',
       'category'],
      dtype='object')

In [43]:
df.ratings_count

0          None
1          None
2            60
3            60
4          None
           ... 
3589624    None
3589625      19
3589626    None
3589627    None
3589628    None
Name: ratings_count, Length: 3589629, dtype: object

In [47]:
df_not_null_ratings_count = df.loc[df['ratings_count'].notnull()]

In [48]:
df_not_null_ratings_count

,podcast_id,itunes_id,slug,itunes_url,title,author,description,average_rating,ratings_count,scraped_at,category
2,f201172864310534ae6b6c4f8ae9b8cc,1000016089,rahdo-talks-through,https://podcasts.apple.com/us/podcast/rahdo-ta...,Rahdo Talks Through,noreply@blogger.com (Richard Ham),"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,2019-07-08 06:15:45.106345,leisure-games
3,f201172864310534ae6b6c4f8ae9b8cc,1000016089,rahdo-talks-through,https://podcasts.apple.com/us/podcast/rahdo-ta...,Rahdo Talks Through,noreply@blogger.com (Richard Ham),"Richard ""Rahdo"" Ham talks (at great length) al...",4.6,60,2019-07-08 06:15:45.106345,leisure
9,3cfef94cfccf27eb504bd7a6c7459636,1000035707,originall,https://podcasts.apple.com/us/podcast/original...,OriginALL,CityPodcast,OriginALL este un show cu oameni inteligenti p...,3.7,3,2019-07-08 08:49:41.523299,business-careers
10,3cfef94cfccf27eb504bd7a6c7459636,1000035707,originall,https://podcasts.apple.com/us/podcast/original...,OriginALL,CityPodcast,OriginALL este un show cu oameni inteligenti p...,3.7,3,2019-07-08 08:49:41.523299,business
13,9fc35a81647e391f93211b5d11973c83,1000081327,galaktacus,https://podcasts.apple.com/us/podcast/galaktac...,Galaktacus,"Jim Kettner, John Mihaly, and Scott Brown",The name Galaktacus originates from a youthful...,5.0,20,2019-07-09 16:22:01.395399,tv-film
...,...,...,...,...,...,...,...,...,...,...,...
3589590,075747aa236dd5e97bfa7834a6bce607,999882879,oakgrovebaptist,https://podcasts.apple.com/us/podcast/oakgrove...,OakGroveBaptist,OakGroveBaptist,We want to help you make a connection - a conn...,4.0,1,2019-07-08 15:28:44.69319,religion-spirituality
3589604,c108b5f6d6972a6e9d900d222a0e6309,999909601,dental-up,https://podcasts.apple.com/us/podcast/dental-u...,Dental Up,"Keating Dental Arts: Dental, Dentistry, Dentist",Dental Up is your source for everything dental...,5.0,21,2019-07-08 03:43:21.393046,education
3589620,3c19556d722b83c643f2d0ff59891ec1,999962739,sarika-d-mehta-podcast,https://podcasts.apple.com/us/podcast/sarika-d...,Sarika D. Mehta » Podcast,Sarika D. Mehta,Awesome interviews with fascinating people.,5.0,2,2019-07-07 20:53:17.582329,arts
3589621,3c19556d722b83c643f2d0ff59891ec1,999962739,sarika-d-mehta-podcast,https://podcasts.apple.com/us/podcast/sarika-d...,Sarika D. Mehta » Podcast,Sarika D. Mehta,Awesome interviews with fascinating people.,5.0,2,2019-07-07 20:53:17.582329,news


In [51]:
reviews_df



,podcast_id,title,content,rating,author_id,created_at
0,52e3d2c4fab4e80a8bb75ad144671d96,Argh,What the heck is this show? And why can’t I de...,1,a23655ce5565f32,2019-05-01 01:24:01+00
1,52e3d2c4fab4e80a8bb75ad144671d96,Miss gross overrated as interviewer.,The Howard stern interview was horrible. I lea...,1,e5d461c5f91c8d2,2019-06-07 14:25:34+00
2,52e3d2c4fab4e80a8bb75ad144671d96,I love Terri and Fresh Air,People get really pissy about this beautiful f...,5,806cc75442c8f3d,2019-05-15 09:42:53+00
3,52e3d2c4fab4e80a8bb75ad144671d96,Fresh Air subscription changes to Code Switch,I’ve been a subscriber to Fresh Air for over a...,4,3e1e7e8417a3e85,2019-05-06 02:39:10+00
4,52e3d2c4fab4e80a8bb75ad144671d96,Climate change,"Hello, my name is Lily Rudman and I am a fifth...",5,7c680144b582c6d,2019-04-18 00:37:51+00
...,...,...,...,...,...,...
5607016,0c06304da3173d464b4487d6040b46d4,Love!,Alex! You are hitting it out of the park with ...,5,25c5550051cf584,2023-03-03 13:07:43+00
5607017,c1b72d56046b06a69cc083a6a5732051,Simple way to get ahead,The discussion on the role of the Export-Impor...,5,750e3e2492a9c13,2023-03-03 16:20:13+00
5607018,95954ad0693967fa1d87dee305cd6a30,Wonderful podcast; ill-fitting intro song,It made no sense in the context at all. Great ...,4,09c2426888b6a94,2023-03-03 15:36:57+00
5607019,2c86f61c1e26ff641a95f5bff6297dfd,Binging like a madman,"My girlfriend turned me onto this podcast, and...",5,fd53bc0a84bd210,2023-03-03 13:29:11+00


In [53]:
reviews_df.podcast_id.nunique()

303911

In [54]:
df.podcast_id.nunique()

2064893